In [1]:
import pandas as pd
import numpy as np
import joblib
import sklearn
# import ensemble
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error,balanced_accuracy_score,recall_score
import random
import lightgbm as lgb

In [2]:
import xgboost as xgb

In [3]:
data = pd.read_parquet("Data/Training/Amex_Campus_Challenge_Train.parquet")

In [4]:
randomlist = random.sample(list(data.customer.unique()),int(0.2*len(data.customer.unique())))

In [5]:
testdata = data[data['customer'].isin(randomlist)]

In [6]:
#select rows whose column value is not in a list
# traindata = data[~data['customer'].isin(randomlist)]
traindata = data

datarec = traindata[traindata['ind_recommended']==1].drop(['ind_recommended'], axis=1)
datanotrec = traindata[traindata['ind_recommended']==0].drop(['ind_recommended'], axis=1)

In [7]:
def confusion_matrix_2col(df, col1, col2):
    return df.groupby([col1, col2]).size().unstack(fill_value=0)

confusion_matrix_2col(traindata, 'ind_recommended', 'activation')

activation,0,1
ind_recommended,,
0,10622736,60173
1,1537226,9843


In [8]:
X_train, X_test, y_train, y_test = train_test_split(traindata.drop(['customer','merchant','activation','ind_recommended'],axis=1), traindata['ind_recommended'], test_size=0.2, random_state=1)
rec = lgb.LGBMClassifier()

In [ ]:
print("Accuracy : ",accuracy_score(y_test, y_pred),"Balanced Accuracy :",balanced_accuracy_score(y_test, y_pred)," recall :",recall_score(y_test, y_pred))

In [ ]:
rec.fit(X_train, y_train)
y_pred = rec.predict(X_test)
# print("mse : ",mean_squared_error(y_test, y_pred))

In [ ]:
traindata.groupby('activation').count()

In [ ]:
nrec = get_model(traindata.drop(traindata[traindata['activation'] == 0].sample(n=10452736).index))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(traindata.drop(['customer','merchant','activation','ind_recommended'],axis=1), traindata[''], test_size=0.2, random_state=1)
rec = ensemble.HistGradientBoostingClassifier(random_state=1)

In [12]:
datanotrec['customer'].groupby(datanotrec['activation']).count()

activation
0    10622736
1       60173
Name: customer, dtype: int64

In [13]:
nrec = get_model(datanotrec.drop(datanotrec[datanotrec['activation'] == 0].sample(n=10452736).index))

Balanced Accuracy : 0.8523036065270284  recall : 0.7761696398519017


In [14]:
a = testdata[['ind_recommended','activation','customer','merchant']]
a.to_csv('d_test.csv', index=False)

In [15]:
try:
    testdata.drop(['rec','nrec','predicted_score'],axis=1,inplace=True)
except:
    pass

In [16]:
testdata

,ind_recommended,activation,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,...,merchant_spend_09,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant
2,0,0,NaN,31.623103,48.837872,19.0,2295.38,47.0,42.0,11.0,...,3912.0,12868.0,0.836364,421.500000,33.780445,0.000000,180.0,1.753009,225178,7439
12,0,0,NaN,227.978605,88.336000,9.0,883.36,10.0,10.0,19.0,...,6482.0,14746.0,0.558140,111.470588,95.394332,6.333333,100.0,2.290935,335462,219596
17,0,0,NaN,63.807500,182.868182,8.0,2011.55,11.0,11.0,5.0,...,NaN,NaN,0.982456,5.850000,97.487533,1.333333,111.0,12.676034,452082,106411
22,1,0,NaN,24.482500,93.407333,7.0,1401.11,15.0,10.0,7.0,...,23053.0,33427.0,0.222222,65.370000,99.873150,2.333333,258.0,18.607740,120246,298378
28,0,0,NaN,326.447500,631.195000,1.0,1262.39,2.0,2.0,2.0,...,24492.0,23859.0,1.000000,33.500000,23.205902,2.333333,260.0,0.484150,113675,31057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12229960,0,1,2.0,336.494865,111.918182,34.0,6155.50,55.0,50.0,8.0,...,208.0,423.0,1.000000,60.000000,54.607556,0.333333,558.0,2.794109,22528,517279
12229962,0,1,NaN,57.289091,NaN,NaN,NaN,NaN,NaN,16.0,...,12944.0,49949.0,1.000000,56.960000,71.475632,3.333333,457.0,1.869187,22660,246363
12229964,0,1,1.0,69.532529,21.116111,9.0,1140.27,54.0,49.0,34.0,...,4643.0,9504.0,0.522727,25.287045,69.105417,7.833333,569.0,0.771669,22764,467964
12229970,0,1,NaN,47.003182,47.904400,22.0,1197.61,25.0,24.0,11.0,...,33603.0,3394.0,0.777778,35.970000,10.267788,1.000000,365.0,0.672902,22926,293661


In [17]:
rec_list = rec.predict_proba(testdata.drop(['ind_recommended','activation','customer','merchant'],axis=1))[:,1]
nrec_list = nrec.predict_proba(testdata.drop(['ind_recommended','activation','customer','merchant'],axis=1))[:,1]

In [18]:
#add column to df 
testdata['rec'] = list(rec_list)
testdata['nrec'] = list(nrec_list)

/var/folders/yy/95rx0nm12jqc32p5svcxxv7h0000gn/T/ipykernel_97051/89867839.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdata['rec'] = list(rec_list)
/var/folders/yy/95rx0nm12jqc32p5svcxxv7h0000gn/T/ipykernel_97051/89867839.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdata['nrec'] = list(nrec_list)


In [26]:
# testdata['predicted_score'] = testdata['rec']*(abs(1-testdata['nrec']))
testdata['predicted_score'] = testdata['rec']-testdata['nrec']

/var/folders/yy/95rx0nm12jqc32p5svcxxv7h0000gn/T/ipykernel_97051/1127479097.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdata['predicted_score'] = testdata['rec']-testdata['nrec']


In [27]:
testdata

,ind_recommended,activation,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,...,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant,rec,nrec,predicted_score
2,0,0,NaN,31.623103,48.837872,19.0,2295.38,47.0,42.0,11.0,...,421.500000,33.780445,0.000000,180.0,1.753009,225178,7439,0.000589,0.005754,-0.005165
12,0,0,NaN,227.978605,88.336000,9.0,883.36,10.0,10.0,19.0,...,111.470588,95.394332,6.333333,100.0,2.290935,335462,219596,0.047908,0.097715,-0.049807
17,0,0,NaN,63.807500,182.868182,8.0,2011.55,11.0,11.0,5.0,...,5.850000,97.487533,1.333333,111.0,12.676034,452082,106411,0.001731,0.001294,0.000437
22,1,0,NaN,24.482500,93.407333,7.0,1401.11,15.0,10.0,7.0,...,65.370000,99.873150,2.333333,258.0,18.607740,120246,298378,0.040346,0.040068,0.000278
28,0,0,NaN,326.447500,631.195000,1.0,1262.39,2.0,2.0,2.0,...,33.500000,23.205902,2.333333,260.0,0.484150,113675,31057,0.103282,0.158452,-0.055169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12229960,0,1,2.0,336.494865,111.918182,34.0,6155.50,55.0,50.0,8.0,...,60.000000,54.607556,0.333333,558.0,2.794109,22528,517279,0.524937,0.571075,-0.046138
12229962,0,1,NaN,57.289091,NaN,NaN,NaN,NaN,NaN,16.0,...,56.960000,71.475632,3.333333,457.0,1.869187,22660,246363,0.836469,0.914568,-0.078099
12229964,0,1,1.0,69.532529,21.116111,9.0,1140.27,54.0,49.0,34.0,...,25.287045,69.105417,7.833333,569.0,0.771669,22764,467964,0.921918,0.962144,-0.040225
12229970,0,1,NaN,47.003182,47.904400,22.0,1197.61,25.0,24.0,11.0,...,35.970000,10.267788,1.000000,365.0,0.672902,22926,293661,0.881019,0.761694,0.119326


In [28]:
testdata[['customer','predicted_score','merchant']].to_csv('d_pred.csv',index=False)

In [22]:
name='trial'

In [125]:
joblib.dump(rec, 'submissions/'+name+'/models/rec.joblib')
joblib.dump(nrec, 'submissions/'+name+'/models/nrec.joblib')

['submissions/80percent_secondprob/models/nrec.joblib']

In [126]:
testdata.predicted_score.mean()

0.0037679773595457318